# Next Tweetscape Experiment: Stream Seeding
Fitering Methods, sorted by (my guess at) technical difficulty

## Metadata methods 
How could we use seed accounts to build a good stream from a small number of accounts that is related to a particular subject. 

These methods will likely be too simple, but it will be interesting to see how close they can get us to something "acceptable" 

These methods are much less complex than the NLP ones. Tools like twemex are massively popular without any complex methods, so it is worth exploring how far they can take us before adding unnecessary complexity. 

#### Top Tweets
* Get the top tweets from each account (relative to themselves) for a certin date range
    * try likes, replies, rts, qts
        * question: does a high number of replies/likes/rts/qts (any one of them significantly) signify that it is more likely they are talking about the subject matter the creator finds interesting? 
        * do these accounts have shitposts that blow up, and then they happen to talk about interesting stuff, or does their interesting stuff itself blow up

#### Top Referenced Tweets
* What tweets are these people replying to, rt'ing, and qt'ing? 
    * do they reference tweets that are of interest to the creator of the stream? 
        * my guess: quoted tweets will be of most interest


#### Referenced Tweets Overlap
* Which tweets are referenced by multiple of the people seeded in the stream? 
    * does this signify anything significant
    * will this actually produce results with such a small number of people? 

#### Referenced Tweets with Follower/Following Overlap
* Does account A quote a tweet by someone that Account B follows (when A and B are both in the seeding of the stream?) 
    * can we use this to grab interesting tweets? 

In [1]:
import os
import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.express as px

from sqlalchemy import create_engine #importing sqlalchemy engine to create engine for the database

from twarc import Twarc2, expansions
from twarc_csv import DataFrameConverter

from data import extract_double_mention, extract_num_mentions, extract_tweet_type, extract_usernames


from dotenv import load_dotenv

load_dotenv()

client = Twarc2(
    consumer_key=os.environ["consumer_key"], 
    consumer_secret=os.environ["consumer_secret"],
    access_token=os.environ["access_token"], 
    access_token_secret=os.environ["access_token_secret"]
)
converter = DataFrameConverter("tweets", allow_duplicates=True)

### First Seeding: CA Abundance Economy
* https://twitter.com/rklau 
* https://twitter.com/anniefryman 
* https://twitter.com/hanlonbt 


In [2]:
## SAVING data locally
from data import save_tweets
usernames = ["rklau", "anniefryman", "hanlonbt"]
group_name = "CA-Abundance-Economy"
# save_tweets(usernames, group_name)

In [3]:
## LOADING DATA
from data import load_data
dfs = load_data(group_name)

combined_tweets = []
combined_refs = []

for username, i_dfs in dfs.items():
    ref_tweets = i_dfs["ref_tweets"]
    tweets = i_dfs["tweets"]
    ref_tweets["author.referencer"] = username
    combined_refs.append(ref_tweets)
    combined_tweets.append(tweets)

tweets = pd.concat(combined_tweets)
ref_tweets = pd.concat(combined_refs)
tweets.shape

for df_ in [tweets, ref_tweets]:
    df_["entities.mentions.usernames"] = df_["entities.mentions"].apply(extract_usernames)
    df_["entities.mentions.num_mentions"] = df_["entities.mentions"].apply(extract_num_mentions)
    df_["entities.mentions.double_mention"] = df_["entities.mentions"].apply(extract_double_mention)
    df_["tweet_type"] = df_.apply(lambda x: extract_tweet_type(x), axis=1)
    df_["tweet_link"] = df_.apply(lambda row: f"https://twitter.com/{row['author.username']}/status/{row.id}", axis=1)
    df_.loc[:, "created_at"] = pd.to_datetime(df_.loc[:, "created_at"], utc=True)
    df_["created_at.hour"] = df_["created_at"].dt.floor('h')

In [4]:
tweets["tweet_type"].unique()

array(['rt', 'self-reply', 'standalone', 'reply', 'standalone,mention',
       'self-reply,mention', 'rt,mention', 'qt', 'qt,mention',
       'qt,self-reply', 'qt,reply', 'qt,self-reply,mention'], dtype=object)

## TimeRange
**define the time range to use to buil all feeds**

* starting with 1 week for good sample size

In [5]:
from datetime import datetime, timezone

def get_time_interval(hours=24):
    """
    ## Get dates for a 24 hour window to pass to twarc2 timeline command

    Return EndTime, StartTime, from current time
    """
    now = datetime.now(timezone.utc)
    (now - timedelta(hours=hours)).isoformat("T")[:-3] + "Z"
    return now.isoformat("T")[:-13]+"Z",  (now - timedelta(hours=hours)).isoformat("T")[:-13] + "Z"

NUM_DAYS = 7

end_time, start_time = get_time_interval(hours=24*NUM_DAYS)
print("start_time: ", start_time, "end_time: ", end_time)


tweets_range = tweets[tweets["created_at"] > start_time]


print("latest tweet in data: ", tweets_range["created_at"].max())
print(tweets_range.shape)

ref_tweets_range = ref_tweets[ref_tweets["created_at"] > start_time]

start_time:  2022-07-11T19:31:05Z end_time:  2022-07-18T19:31:05Z
latest tweet in data:  2022-07-15 11:21:04+00:00
(54, 81)


## Tweet Type Distribution in Range

In [6]:
tweets_range["author.username"].value_counts()

rklau          30
anniefryman    20
hanlonbt        4
Name: author.username, dtype: int64

In [7]:
tweets_range[tweets_range["author.username"] == "anniefryman"]["created_at"]

0    2022-07-15 06:13:32+00:00
1    2022-07-15 02:15:10+00:00
2    2022-07-15 02:10:15+00:00
3    2022-07-15 02:06:07+00:00
4    2022-07-15 02:05:12+00:00
5    2022-07-15 02:02:37+00:00
6    2022-07-15 01:15:45+00:00
7    2022-07-15 01:11:34+00:00
8    2022-07-14 18:02:06+00:00
9    2022-07-14 15:35:29+00:00
10   2022-07-14 15:35:29+00:00
11   2022-07-14 15:29:32+00:00
12   2022-07-14 15:22:25+00:00
13   2022-07-14 00:10:24+00:00
14   2022-07-12 20:16:16+00:00
15   2022-07-12 19:24:00+00:00
16   2022-07-12 19:07:01+00:00
17   2022-07-12 18:51:50+00:00
18   2022-07-12 03:40:54+00:00
19   2022-07-12 03:33:43+00:00
Name: created_at, dtype: datetime64[ns, UTC]

In [8]:
tweets_range["tweet_type"].value_counts()

reply                 14
self-reply            12
standalone             8
self-reply,mention     6
rt                     4
standalone,mention     4
rt,mention             4
qt                     2
Name: tweet_type, dtype: int64

In [9]:
## user tweets distribution
user = "anniefryman"
df_ = tweets_range[tweets_range["author.username"] == user]
df_["tweet_type"].value_counts()

self-reply            7
standalone            5
reply                 4
standalone,mention    2
rt                    1
self-reply,mention    1
Name: tweet_type, dtype: int64

## Standalone Tweet Feed

In [10]:
import requests
from IPython.display import display, Markdown

class Tweet(object):
    """
    This class is used to display tweets in feeds in the notebook
    """
    def __init__(self, s, embed_str=False):
        if not embed_str:
            # Use Twitter's oEmbed API
            # https://dev.twitter.com/web/embedded-tweets
            api = 'https://publish.twitter.com/oembed?url={}'.format(s)
            response = requests.get(api)
            self.text = response.json()["html"]
        else:
            self.text = s

    def _repr_html_(self):
        return self.text
    
def display_feed(tweet_links): 
    feed = ""

    for link in tweet_links:
        feed += Tweet(link)._repr_html_()

    display(Markdown(feed))

In [11]:
## Get standalone tweets from these users
tweets_stand = tweets_range[(tweets_range["tweet_type"] == "standalone") | (tweets_range["tweet_type"] == "standalone,mention")]

## See how many tweets from each user
tweets_stand["author.username"].value_counts()

anniefryman    7
rklau          5
Name: author.username, dtype: int64

In [12]:
## Show feed of standalone tweets
display_feed(tweets_stand.tweet_link.tolist())

<blockquote class="twitter-tweet"><p lang="en" dir="ltr">A clear, discrete, impactful civil rights policy that President Biden could champion is requiring that any National Historic Designation under consideration by the Department of the Interior must include a fair housing analysis, proving the designation does not run afoul of HUD.</p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1547764186402369537?ref_src=twsrc%5Etfw">July 15, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">I talked to <a href="https://twitter.com/urbenschneider?ref_src=twsrc%5Etfw">@urbenschneider</a> at <a href="https://twitter.com/sfexaminer?ref_src=twsrc%5Etfw">@sfexaminer</a> about the incoherent architecture and pleasant landscaping of St. Francis Wood neighborhood, and how historical preservation can be a politically correct way to maintain (or exacerbate!) racial segregation.<a href="https://t.co/cKURboCOX8">https://t.co/cKURboCOX8</a></p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1547750688377081856?ref_src=twsrc%5Etfw">July 15, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">I recently got wired into a loose social community of people, through one chance friend/introduction, of ~150 people or so in San Francisco. I&#39;d say about half of the folks I&#39;ve met or talked to so far moved to the city during COVID from elsewhere in the country.</p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1547605712192212995?ref_src=twsrc%5Etfw">July 14, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">...turns out! <a href="https://t.co/FYXl4eQT9h">https://t.co/FYXl4eQT9h</a></p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1547602422989611011?ref_src=twsrc%5Etfw">July 14, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="zxx" dir="ltr"><a href="https://t.co/2n39Ypq33W">pic.twitter.com/2n39Ypq33W</a></p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1547372906224943105?ref_src=twsrc%5Etfw">July 14, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">I talked to <a href="https://twitter.com/dustingardiner?ref_src=twsrc%5Etfw">@dustingardiner</a> at the <a href="https://twitter.com/sfchronicle?ref_src=twsrc%5Etfw">@sfchronicle</a> recently about what needs to be done if we want Senate Bill 9 to *actually* work at the enormous scale California&#39;s State Legislature promised.<a href="https://t.co/LAB2emiKhN">https://t.co/LAB2emiKhN</a></p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1546930348042248198?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Spent the last five days outdoors in the 105° Southern Arizona desert <br><br>Tonight my body is home sweet home 🌁 <a href="https://t.co/cHttx6mxjB">pic.twitter.com/cHttx6mxjB</a></p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1546699297495822337?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Just booked my first international travel in almost 3 years. You may be wondering if, when <a href="https://twitter.com/Delta?ref_src=twsrc%5Etfw">@Delta</a> automatically verified my uploaded proof of vaccination, was I momentarily overwhelmed?<br><br>Yes. Yes I was. <a href="https://t.co/ljQO1RGS3d">pic.twitter.com/ljQO1RGS3d</a></p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1547318706681720832?ref_src=twsrc%5Etfw">July 13, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Last week was my last week at the state of California. Serving as the state’s Chief Technology Innovation Officer for the last 18 months has been a privilege, and I’ll forever be grateful for the opportunity to serve the people of California during a time of such tremendous need.</p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1547001132047667200?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">a propos of nothing, pretty sure I remember a <a href="https://twitter.com/bbcdoctorwho?ref_src=twsrc%5Etfw">@bbcdoctorwho</a> episode that used audio files to brainwash an entire planet</p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1546618236862545920?ref_src=twsrc%5Etfw">July 11, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">someone file a FOIA request to make sure that the composer of this little ditty isn&#39;t the NASA Public Information Officer&#39;s nephew and this isn&#39;t all just a royalty ploy</p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1546615049359265792?ref_src=twsrc%5Etfw">July 11, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="zxx" dir="ltr"><a href="https://t.co/0gJJIPqOIm">https://t.co/0gJJIPqOIm</a> <a href="https://t.co/xRL9AoAyWk">pic.twitter.com/xRL9AoAyWk</a></p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1546610929986465792?ref_src=twsrc%5Etfw">July 11, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>


## All Tweets, filter by like count
This will skew highly towards more popular accounts

* only showing 1 tweet per group with the same conversation_id (threads)

**I'm leaving rt's for now, but that could be annoying...**

In [13]:
minimum_likes = 10
tweets_min_likes = tweets_range[tweets_range["public_metrics.like_count"] > minimum_likes]
tweets_min_likes = tweets_range[tweets_range["public_metrics.like_count"] > minimum_likes]
print(tweets_min_likes["tweet_type"].value_counts())
tweets_min_likes["author.username"].value_counts()

self-reply            11
standalone             6
self-reply,mention     5
rt                     3
standalone,mention     3
rt,mention             3
reply                  1
Name: tweet_type, dtype: int64


rklau          17
anniefryman    13
hanlonbt        2
Name: author.username, dtype: int64

In [14]:
annie = tweets_min_likes[tweets_min_likes["author.username"] == "anniefryman"]
rklau = tweets_min_likes[tweets_min_likes["author.username"] == "rklau"]

In [15]:
rklau[rklau["tweet_type"]=="reply"].conversation_id.unique().shape

(1,)

In [16]:
for link, conv_id in zip(rklau[rklau["tweet_type"]=="reply"].tweet_link.tolist(), rklau[rklau["tweet_type"]=="reply"].conversation_id.tolist()):
    print(link, conv_id)

https://twitter.com/rklau/status/1547319569823346693 1547319131585798144


In [17]:
print(tweets_min_likes.shape)
tweets_min_likes.conversation_id.unique().shape

(32, 81)


(16,)

In [18]:
def get_earliest(group):
    earliest = group["created_at"].min()
    return group[group["created_at"] == group["created_at"].min()].iloc[:1] # randomly choose if there is more than one, for now. breaks for threads tweeted all at once...
                 
starts = tweets_min_likes.groupby("conversation_id").apply(lambda x: get_earliest(x))


In [19]:
display_feed(tweets_min_likes.groupby("conversation_id").apply(lambda x: get_earliest(x)).tweet_link.tolist())

<blockquote class="twitter-tweet"><p lang="en" dir="ltr">&quot;The most important question is: How can we guarantee that every person has a permanent, safe, accessible home?&quot; <a href="https://twitter.com/hashtag/housingisahumanright?src=hash&amp;ref_src=twsrc%5Etfw">#housingisahumanright</a><br><br>by: <a href="https://twitter.com/hanlonbt?ref_src=twsrc%5Etfw">@hanlonbt</a> <a href="https://twitter.com/resnikoff?ref_src=twsrc%5Etfw">@resnikoff</a> <a href="https://twitter.com/jbwashing?ref_src=twsrc%5Etfw">@jbwashing</a> <a href="https://twitter.com/taraghuveer?ref_src=twsrc%5Etfw">@taraghuveer</a> | <a href="https://twitter.com/thenation?ref_src=twsrc%5Etfw">@thenation</a> <a href="https://t.co/tAfbCERRzW">https://t.co/tAfbCERRzW</a></p>&mdash; Family Promise (@fpnational) <a href="https://twitter.com/fpnational/status/1546256700981780480?ref_src=twsrc%5Etfw">July 10, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">someone file a FOIA request to make sure that the composer of this little ditty isn&#39;t the NASA Public Information Officer&#39;s nephew and this isn&#39;t all just a royalty ploy</p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1546615049359265792?ref_src=twsrc%5Etfw">July 11, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Spent the last five days outdoors in the 105° Southern Arizona desert <br><br>Tonight my body is home sweet home 🌁 <a href="https://t.co/cHttx6mxjB">pic.twitter.com/cHttx6mxjB</a></p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1546699297495822337?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">I talked to <a href="https://twitter.com/dustingardiner?ref_src=twsrc%5Etfw">@dustingardiner</a> at the <a href="https://twitter.com/sfchronicle?ref_src=twsrc%5Etfw">@sfchronicle</a> recently about what needs to be done if we want Senate Bill 9 to *actually* work at the enormous scale California&#39;s State Legislature promised.<a href="https://t.co/LAB2emiKhN">https://t.co/LAB2emiKhN</a></p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1546930348042248198?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Last week was my last week at the state of California. Serving as the state’s Chief Technology Innovation Officer for the last 18 months has been a privilege, and I’ll forever be grateful for the opportunity to serve the people of California during a time of such tremendous need.</p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1547001132047667200?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">I still have this kind note and challenge coin you gave me last year; it captures your leadership style so well! ❤️ It was an honor working with you in CA &amp; watching your team blossom.🏵 Thank you for everything &amp; excited to hear about your next adventure!<a href="https://twitter.com/hashtag/OneTeam?src=hash&amp;ref_src=twsrc%5Etfw">#OneTeam</a>🐻 <a href="https://t.co/h540BjlWdZ">pic.twitter.com/h540BjlWdZ</a></p>&mdash; AngieQ (@abquirarte) <a href="https://twitter.com/abquirarte/status/1547051132643528704?ref_src=twsrc%5Etfw">July 13, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Just booked my first international travel in almost 3 years. You may be wondering if, when <a href="https://twitter.com/Delta?ref_src=twsrc%5Etfw">@Delta</a> automatically verified my uploaded proof of vaccination, was I momentarily overwhelmed?<br><br>Yes. Yes I was. <a href="https://t.co/ljQO1RGS3d">pic.twitter.com/ljQO1RGS3d</a></p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1547318706681720832?ref_src=twsrc%5Etfw">July 13, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">amazing! (Hope you and the car are ok!)</p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1547319569823346693?ref_src=twsrc%5Etfw">July 13, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">If you have ever wondered if you can have a massive impact, check out the great work of <a href="https://twitter.com/rklau?ref_src=twsrc%5Etfw">@rklau</a> and the incredible civil servants at CA Dept of Tech <a href="https://t.co/vDbzzVvdrw">https://t.co/vDbzzVvdrw</a></p>&mdash; dj patil (@dpatil) <a href="https://twitter.com/dpatil/status/1547067260929560576?ref_src=twsrc%5Etfw">July 13, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Rick has done some absolutely phenomenal work in California, and is a great example of the good that can come from smart technologists going to work in government. <a href="https://t.co/sQW3gyDINT">https://t.co/sQW3gyDINT</a></p>&mdash; Mike Sager 🇺🇦 (@msager) <a href="https://twitter.com/msager/status/1547061923493150721?ref_src=twsrc%5Etfw">July 13, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">...turns out! <a href="https://t.co/FYXl4eQT9h">https://t.co/FYXl4eQT9h</a></p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1547602422989611011?ref_src=twsrc%5Etfw">July 14, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Most are in their mid- to late-  20s and looking to build their lives in the city, at least medium term, and have been really proactive about building community here: starting coliving spaces, open community spaces, etc.<br><br>It&#39;s neat to see narrative violations in the wild.</p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1547605713412796421?ref_src=twsrc%5Etfw">July 14, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">We play too many shell games in planning. Let&#39;s just actually regulate impacts and buy/build affordable housing. <a href="https://t.co/xMXoQxKUzO">https://t.co/xMXoQxKUzO</a></p>&mdash; M. Nolan Gray (@mnolangray) <a href="https://twitter.com/mnolangray/status/1547434964094013440?ref_src=twsrc%5Etfw">July 14, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">I talked to <a href="https://twitter.com/urbenschneider?ref_src=twsrc%5Etfw">@urbenschneider</a> at <a href="https://twitter.com/sfexaminer?ref_src=twsrc%5Etfw">@sfexaminer</a> about the incoherent architecture and pleasant landscaping of St. Francis Wood neighborhood, and how historical preservation can be a politically correct way to maintain (or exacerbate!) racial segregation.<a href="https://t.co/cKURboCOX8">https://t.co/cKURboCOX8</a></p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1547750688377081856?ref_src=twsrc%5Etfw">July 15, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">A clear, discrete, impactful civil rights policy that President Biden could champion is requiring that any National Historic Designation under consideration by the Department of the Interior must include a fair housing analysis, proving the designation does not run afoul of HUD.</p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1547764186402369537?ref_src=twsrc%5Etfw">July 15, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">One way to keep cars from using bus lanes as express or passing lanes is grass.  Buses are wide enough to straddle the grass, while cars can&#39;t. <a href="https://t.co/ODQZECUC2z">https://t.co/ODQZECUC2z</a> <a href="https://t.co/n6ZL134ZrS">pic.twitter.com/n6ZL134ZrS</a></p>&mdash; Alfred Twu (@alfred_twu) <a href="https://twitter.com/alfred_twu/status/1536834900598341634?ref_src=twsrc%5Etfw">June 14, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>


## All Tweets, filter by like percentile
Shows top N percentile of tweets by like count for each user

In [22]:
npercentile = 0.75

def filter_by_quantile(group):
    thresh = group["public_metrics.like_count"].quantile(0.75)
    return group[group["public_metrics.like_count"] > thresh]

tweets_quantile = tweets_range.groupby(["author.username"]).apply(lambda x: filter_by_quantile(x))

print(tweets_range.shape)
tweets_quantile.shape

(54, 81)


(14, 81)

In [21]:
display_feed(tweets_quantile.tweet_link.tolist())

<blockquote class="twitter-tweet"><p lang="en" dir="ltr">One way to keep cars from using bus lanes as express or passing lanes is grass.  Buses are wide enough to straddle the grass, while cars can&#39;t. <a href="https://t.co/ODQZECUC2z">https://t.co/ODQZECUC2z</a> <a href="https://t.co/n6ZL134ZrS">pic.twitter.com/n6ZL134ZrS</a></p>&mdash; Alfred Twu (@alfred_twu) <a href="https://twitter.com/alfred_twu/status/1536834900598341634?ref_src=twsrc%5Etfw">June 14, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">What this means: <br><br>If you want to become a national historic district, you have to prove that it won’t promote or maintain racial segregation<br><br>California should do this too</p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1547766692893233155?ref_src=twsrc%5Etfw">July 15, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">A clear, discrete, impactful civil rights policy that President Biden could champion is requiring that any National Historic Designation under consideration by the Department of the Interior must include a fair housing analysis, proving the designation does not run afoul of HUD.</p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1547764186402369537?ref_src=twsrc%5Etfw">July 15, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">I talked to <a href="https://twitter.com/dustingardiner?ref_src=twsrc%5Etfw">@dustingardiner</a> at the <a href="https://twitter.com/sfchronicle?ref_src=twsrc%5Etfw">@sfchronicle</a> recently about what needs to be done if we want Senate Bill 9 to *actually* work at the enormous scale California&#39;s State Legislature promised.<a href="https://t.co/LAB2emiKhN">https://t.co/LAB2emiKhN</a></p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1546930348042248198?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Spent the last five days outdoors in the 105° Southern Arizona desert <br><br>Tonight my body is home sweet home 🌁 <a href="https://t.co/cHttx6mxjB">pic.twitter.com/cHttx6mxjB</a></p>&mdash; fry (@anniefryman) <a href="https://twitter.com/anniefryman/status/1546699297495822337?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">We play too many shell games in planning. Let&#39;s just actually regulate impacts and buy/build affordable housing. <a href="https://t.co/xMXoQxKUzO">https://t.co/xMXoQxKUzO</a></p>&mdash; M. Nolan Gray (@mnolangray) <a href="https://twitter.com/mnolangray/status/1547434964094013440?ref_src=twsrc%5Etfw">July 14, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">If you have ever wondered if you can have a massive impact, check out the great work of <a href="https://twitter.com/rklau?ref_src=twsrc%5Etfw">@rklau</a> and the incredible civil servants at CA Dept of Tech <a href="https://t.co/vDbzzVvdrw">https://t.co/vDbzzVvdrw</a></p>&mdash; dj patil (@dpatil) <a href="https://twitter.com/dpatil/status/1547067260929560576?ref_src=twsrc%5Etfw">July 13, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">amazing! (Hope you and the car are ok!)</p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1547319569823346693?ref_src=twsrc%5Etfw">July 13, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Onward. 🙏🏻</p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1547001844236398593?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Your state almost certainly has a digital service or its equivalent, or you could give <a href="https://twitter.com/USDS?ref_src=twsrc%5Etfw">@USDS</a> a look. Your communities, your state, your country need you.</p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1547001729438269440?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">In the meantime, if you currently work in tech and wonder whether you should consider a tour of duty in government: yes, you should. Volunteer with <a href="https://twitter.com/USDResponse?ref_src=twsrc%5Etfw">@USDResponse</a>. Give <a href="https://twitter.com/TechTalentProj?ref_src=twsrc%5Etfw">@TechTalentProj</a> a call. Reach out to <a href="https://twitter.com/codeforamerica?ref_src=twsrc%5Etfw">@codeforamerica</a>.</p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1547001672110469120?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">To everyone who supported me in this chapter of my career: thank you. Every time one of you showed me your Digital Covid Vaccination Record, you made my year. ❤️<br><br>I don’t have news to share about what’s next – that’s for another day.</p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1547001594482372608?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">I started my work at the state as a volunteer – asked to help in the early days of the state’s COVID-19 vaccination campaign. For six weeks, I worked 16 hour days, seven days a week, seeing first-hand how quickly the state could execute in the midst of a crisis.</p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1547001394615291905?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Last week was my last week at the state of California. Serving as the state’s Chief Technology Innovation Officer for the last 18 months has been a privilege, and I’ll forever be grateful for the opportunity to serve the people of California during a time of such tremendous need.</p>&mdash; Rick Klau (@rklau) <a href="https://twitter.com/rklau/status/1547001132047667200?ref_src=twsrc%5Etfw">July 12, 2022</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
